In [4]:
#Data Analyses Libraries
import pandas as pd                
import numpy as np    
from urllib.request import urlopen
import json
import glob
import os

#Importing Data plotting libraries
import matplotlib.pyplot as plt     
import plotly.express as px       
import plotly.offline as py       
import seaborn as sns             
import plotly.graph_objects as go 
from plotly.subplots import make_subplots
import matplotlib.ticker as ticker
import matplotlib.animation as animation

#Other Miscallaneous Libraries
import warnings
warnings.filterwarnings('ignore')
from IPython.display import HTML
import matplotlib.colors as mc
import colorsys
from random import randint
import re
import csv
import seaborn as sns

In [5]:
#Reading the cumulative cases dataset
covid_cases = pd.read_csv('tesing_dataset.csv')

#Viewing the dataset
covid_cases.head()
df_out = covid_cases.describe()
df_out.to_csv("output.csv")

In [6]:
corr_data = covid_cases.drop(['fips','state','county'], axis=1)
corr = corr_data.corr()
print(corr)
corr.to_csv("relations.csv")

                     cases    deaths       lat       lon  total_population  \
cases             1.000000  0.779383 -0.055813  0.016699          0.745575   
deaths            0.779383  1.000000 -0.005333  0.057830          0.631016   
lat              -0.055813 -0.005333  1.000000 -0.268441         -0.050302   
lon               0.016699  0.057830 -0.268441  1.000000         -0.015284   
total_population  0.745575  0.631016 -0.050302 -0.015284          1.000000   
...                    ...       ...       ...       ...               ...   
min_temp_15d_avg  0.039270  0.029321 -0.513008  0.188313          0.041801   
dewpoint_3d_avg  -0.001068  0.011048 -0.422390  0.294492         -0.001186   
dewpoint_5d_avg   0.000612  0.011661 -0.433742  0.302403         -0.001672   
dewpoint_10d_avg  0.004066  0.012657 -0.450895  0.314511         -0.002895   
dewpoint_15d_avg  0.007168  0.013515 -0.459428  0.320485         -0.004167   

                  area_sqmi  population_density_per_sqmi  num_d

In [7]:
#column names with high correlation
upper_tri = corr.where(np.triu(np.ones(corr.shape),k=1).astype(np.bool))
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.7)]
print(to_drop)

['deaths', 'total_population', 'num_deaths', 'average_number_of_physically_unhealthy_days', 'average_number_of_mentally_unhealthy_days', 'percent_smokers', 'num_alcohol_impaired_driving_deaths', 'num_driving_deaths', 'num_chlamydia_cases', 'teen_birth_rate', 'num_uninsured', 'num_primary_care_physicians', 'num_dentists', 'num_mental_health_providers', 'num_some_college', 'population', 'num_unemployed_CHR', 'labor_force', 'percent_children_in_poverty', 'twentieth_percentile_income', 'num_single_parent_households_CHR', 'num_households_CHR', 'num_associations', 'annual_average_violent_crimes', 'num_injury_deaths', 'injury_death_rate', 'severe_housing_cost_burden', 'num_workers_who_drive_alone', 'num_deaths_2', 'age_adjusted_death_rate', 'num_deaths_3', 'child_mortality_rate', 'num_deaths_4', 'infant_mortality_rate', 'percent_frequent_physical_distress', 'percent_frequent_mental_distress', 'num_hiv_cases', 'num_food_insecure', 'percent_food_insecure', 'num_limited_access', 'num_drug_overdo

In [8]:
x=corr[corr < 1].unstack().transpose().sort_values( ascending=False).drop_duplicates()
y=pd.DataFrame(x)
y.columns=["percent"]
y=y[abs(y['percent'])>0.8]
y.to_csv('highest-correlation.csv')

In [9]:
percent_missing = covid_cases.isnull().sum() * 100 / len(covid_cases)
missing_value_df = pd.DataFrame({'column_name': covid_cases.columns,
                                 'percent_missing': percent_missing})
missing_df = missing_value_df[missing_value_df['percent_missing']>15]
missing_df.to_csv("missing_values.csv")

In [ ]:
sns.pairplot(corr_data, kind="scatter")
plt.show()

In [ ]:
corr_data = covid_cases.drop(['fips','state','county'], axis=1)

#Converting the dataset to the correlation function
corr = corr_data.corr()

#Plotting a heatmap

def heatmap(x, y, size,color):
    fig, ax = plt.subplots(figsize=(20,10))
    
    # Mapping from column names to integer coordinates
    x_labels = corr_data.columns
    y_labels = corr_data.columns
    x_to_num = {p[1]:p[0] for p in enumerate(x_labels)} 
    y_to_num = {p[1]:p[0] for p in enumerate(y_labels)} 
    
    n_colors = 256 # Use 256 colors for the diverging color palette
    palette = sns.cubehelix_palette(n_colors) # Create the palette
    color_min, color_max = [-1, 1] # Range of values that will be mapped to the palette, i.e. min and max possible correlation

    def value_to_color(val):
        val_position = float((val - color_min)) / (color_max - color_min) # position of value in the input range, relative to the length of the input range
        ind = int(val_position * (n_colors - 1)) # target index in the color palette
        return palette[ind]

    
    ax.scatter(
    x=x.map(x_to_num),
    y=y.map(y_to_num),
    s=size * 1000,
    c=color.apply(value_to_color), # Vector of square color values, mapped to color palette
    marker='s')
    
    # Show column labels on the axes
    ax.set_xticks([x_to_num[v] for v in x_labels])
    ax.set_xticklabels(x_labels, rotation=30, horizontalalignment='right')
    ax.set_yticks([y_to_num[v] for v in y_labels])
    ax.set_yticklabels(y_labels)
    
    
    ax.set_xticks([t + 0.5 for t in ax.get_xticks()], minor=True)
    ax.set_yticks([t + 0.5 for t in ax.get_yticks()], minor=True)
    
    ax.set_xlim([-0.5, max([v for v in x_to_num.values()]) + 0.5]) 
    ax.set_ylim([-0.5, max([v for v in y_to_num.values()]) + 0.5])
    
corr = pd.melt(corr.reset_index(), id_vars='index') 
ccorr_data = covid_cases.drop(['fips','state','county'], axis=1)orr.columns = ['x', 'y', 'value']
heatmap(x=corr['x'],y=corr['y'],size=corr['value'].abs(),color=corr['value'])

In [ ]:
df = covid_cases.drop(['date_recorded','date_stay_at_home_announced','date_stay_at_home_effective'],axis=1)
fig = px.box(df, y=”fare_amount”)
fig.show()

In [ ]:
# finding outlierws
covid_cases= covid_cases.drop(['precip_flag','station_name'],axis=1)
def find_outliers_IQR(df):
    q1=df.quantile(0.25)
    q3=df.quantile(0.75)
    IQR=q3-q1
    outliers = df[((df<(q1-1.5*IQR)) | (df>(q3+1.5*IQR)))]
    return outliers
for i in covid_cases.columns:
    print(i,end="")
    print(find_outliers_IQR(covid_cases[i]))